<div style="border:solid green 2px; padding: 20px">
<b>Привет, Светлана!</b>

Меня зовут Александр, и я буду проверять твой проект. Предлагаю общаться на «ты» :) Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать data science. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - **пожалуйста, не перемещай, не изменяй и не удаляй их**. Увидев у тебя ошибку, в первый раз я лишь укажу на ее наличие и дам тебе возможность самой найти и исправить ее. На реальной работе твой начальник будет поступать так же, а я пытаюсь подготовить тебя именно к работе аналитиком. Но если ты пока не справишься с такой задачей - при следующей проверке я дам более точную подсказку. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера ❌:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b> Так выделены небольшие замечания. Я надеюсь, что их ты тоже учтешь - твой проект от этого станет только лучше. Но настаивать на их отработке не буду.

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом.** Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [19]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [22]:
# data = pd.read_csv('./toxic_comments.csv')
data = pd.read_csv('/content/toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Огонь, данные на месте:)</div>

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Удалим колонку с продублированным индексом.

In [3]:
data.drop(columns=['Unnamed: 0'], inplace=True)

Перед обучением моделей текст необходимо обработать: избавиться от стоп-слов и знаков препинания, все слова привести к нижнему регистру и начальной форме, преобразовать в векторы.

In [4]:
# стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# очистка текста
def clear_text(text, stopwords):
    words = re.sub(r'[^a-z]', ' ', text.lower()).split()
    words_without_stopwords = [word for word in words if word not in stopwords]

    return ' '.join(words_without_stopwords)

In [6]:
# лемматизатор
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):

    tokens = word_tokenize(text)                                   # токенизация
    lemm_list = [lemmatizer.lemmatize(token) for token in tokens]  # лемматизация
    lemm_text = ' '.join(lemm_list)

    return lemm_text

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>
    
Так же можешь попробовать использовать spacy:
    
```python
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_spacy(text):
    return " ".join([token.lemma_ for token in nlp(text)])

data['lemm_spacy'] = data['text'].progress_apply(lemmatize_spacy)     
```

Вот так можно следить за выполнением функции (как раз выше его использовал)

```python
    
from tqdm import tqdm
tqdm.pandas()
data['text'] = data['text'].progress_apply(lemmatize_text) 
```
Где *lemmatize_text* - функция

    
P.S Кстати можно ускорить spacy ( раз в 5 +-)
    
```python
new_corpus = []

for doc in tqdm(nlp.pipe(data['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(data['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))
    
data['lemm_spacy_new'] = new_corpus   
    
    
```
    
Для больших объемов текста ```SpaCy``` рекомендует использовать ```nlp.pipe```, который может работать в пакетах ```batch_size```(Допустим ```batch_size=64```, тогда все наши данные делятся на 64 пакета (как прям в фолдах, только там мы пишем число на сколько поделить наши данные (3/5/10 частей),а тут пишем, сколько данных отдать в обработку)  и имеет встроенную поддержку многопроцессорной обработки ```n_process``` (аналог ```n_jobs``` для GridSearchCV).

Кроме того, нужно убедиться, что мы отключили все элементы конвейера, которые мы не планируем использовать, поскольку они просто потратят время на обработку. Если мы выполняем только лемматизацию, то необходимо передать ```disable=["parser", "ner"]``` к ```nlp.pipe```.
    
    
P.S.S Если запустишь ```nlp.pipe```, то первое время нужно подождать прежде, чем начнется обработка 
</div>

In [8]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
# проверка функций
text = data['text'][3]
print(text)
print()
lemmatize_text(clear_text(text, stopwords))

"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.

There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "



'make real suggestion improvement wondered section statistic later subsection type accident think reference may need tidying exact format ie date format etc later one else first preference formatting style reference want please let know appears backlog article review guess may delay reviewer turn listed relevant form eg wikipedia good article nomination transport'

In [10]:
%%time

data['lemm_text'] = data['text'].apply(lambda x: lemmatize_text(clear_text(x, stopwords)))
data.head()

CPU times: user 1min 11s, sys: 269 ms, total: 1min 12s
Wall time: 1min 16s


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wondered sect...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


Деление на выборки:

In [11]:
data_train, data_test = train_test_split(data, test_size=0.25, random_state=123)
data_train.shape, data_test.shape

((119469, 3), (39823, 3))

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично;) данные готовы

 Так же можно использовать stratify=target, некоторые задачи классификации не имеют сбалансированного количества примеров для каждой метки класса. Таким образом, желательно разделить набор данных на обучающие и тестовые наборы таким образом, чтобы сохранить те же пропорции примеров в каждом классе, которые наблюдаются в исходном наборе данных.


</div>

In [12]:
# тренировочная выборка
corpus_train = data_train['lemm_text'].values             # корпус текстов
target_train = data_train['toxic'].values                 # метки для классификации

# тестовая выборка
corpus_test = data_test['lemm_text'].values               # корпус текстов
target_test = data_test['toxic'].values                   # метки для классификации

Применение векторного представления:

In [13]:
count_tf_idf = TfidfVectorizer()

In [14]:
%%time

count_tf_idf.fit(corpus_train)

train_tf_idf = count_tf_idf.transform(corpus_train)
test_tf_idf = count_tf_idf.transform(corpus_test)

CPU times: user 11 s, sys: 141 ms, total: 11.2 s
Wall time: 11.3 s


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, молодец, верно используешь Tfidf.
    
    
Совет: Внутри кросс-валидации происходит разбиение выборки на train и valid. Однако, в таком случае векторизатор обучен на всей выборке(train), а это не совсем корректно. Чтобы избежать это можно воспользоваться Pipeline:
    
```python
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=42)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'logreg__C': [1,2,6]
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(train_features, train_targets)
  
    
```
    
Это просто каркас, можешь сам выбрать какие параметры использовать для подбора:) 
    
+  https://runebook.dev/ru/docs/scikit_learn/modules/generated/sklearn.model_selection.halvinggridsearchcv - тут про HalvingGridSearchCV
    
+  https://www.rupython.com/python-sklearn-pipeline-pipeline-28301.html - про pipeline
+  https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

</div>

## Обучение

Попробуем 2 различных алгоритма для решения поставленной задачи: *Логистическую регрессию* и *Случайный лес*.

In [15]:
%%time

# логистическая регрессия
model = LogisticRegression(class_weight='balanced', random_state=123, max_iter=500)
params = {'solver': ['newton-cg', 'liblinear', 'saga']}

grid = GridSearchCV(model, params, cv=5, scoring='f1')
grid.fit(train_tf_idf, target_train)
lr_model = grid.best_estimator_

print(grid.best_params_)
print(grid.best_score_)

{'solver': 'newton-cg'}
0.750300783546443
CPU times: user 3min 48s, sys: 30.1 s, total: 4min 18s
Wall time: 3min 48s


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Хорошо, но у нас дсбаланс, поэтому лучше использовать StratifiedKFold


+  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
    
    
Метод stratified k-Fold — это метод k-Fold, использующий стратификацию при разбиении на фолды: каждый фолд содержит примерно такое же соотношение классов, как и всё исходное множество (подробнее о стратификации говорилось выше). Такой подход может потребоваться в случае, например, очень несбалансированного соотношения классов, когда при обычном random split некоторые фолды могут либо вообще не содержать семплов каких-то классов, либо содержать их слишком мало (найти оригинал можно тут)
+  https://academy.yandex.ru/handbook/ml/article/kross-validaciya
    
</div>



In [18]:
%%time

# случайный лес
model = RandomForestClassifier(class_weight='balanced', random_state=123)
params = {'n_estimators': [200, 300, 400],
          'max_depth': [7, 9, 11]}

grid = GridSearchCV(model, params, cv=3, scoring='f1')
grid.fit(train_tf_idf, target_train)
rf_model = grid.best_estimator_

print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 11, 'n_estimators': 400}
0.3780097561427285
CPU times: user 30min 49s, sys: 1.88 s, total: 30min 51s
Wall time: 30min 59s


Логистическая регрессия показала себя лучше для решения поставленной задачи. Проверим работоспособность модели на тестовой выборке:

In [20]:
preds = lr_model.predict(test_tf_idf)
f1_score(target_test, preds)

0.7504366812227075

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Получился хороший результат:) 
    
</div>



## Выводы

В рамках данного проекта предполагалось создать модель, позволяющую с достаточной точностью определять токсичные комментарии. Перед обучением текстовые данные были подготовлены: очищены от стоп-слов и ненужных символов, все слова лемматизированы и переведены в векторых формат (tf-idf).

Наилучший результат показала модель, обученная алгоритмом логистической регрессии. И на тестовой выборке, и на тренировочной удалось получить F1-score больше 0.75.

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> 

Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множество реализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 
    
</div>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>
<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Светлана, спасибо за хороший проект!!! Я готов принять работу, но хочу убедиться, что тебе все понятно.<br>
Если есть какие либо вопросы я с удовольствием на них отвечу:)</div>


<div class="alert alert-info"> <b>Комментарий студента:</b> Спасибо за очень содержательное ревью и множество полезных ссылок! Я обязательно изучу все и вернусь к ним, когда буду оформлять проект для github, сейчас, к сожалению, время поджимает (. Есть один вопрос, который хотела задать: я пыталась еще обучить модель с испльзованием алгоритма SVC, метода опорных векторов, но не получилось (ооочень долго обучалась). Почему так и что я могу сделать для ускорения процесса, если компьютер не такой мощный, как хотелось бы? Слышала, что этот метод хорошо может себя показать в подобных задачах. Код использовала тот же, что и для других моделей и пыталась подобрать параметр kernel. </div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Думаю лучше всего использовать kernel='linear', а подобрать параметр C ( он отвечает за регуляризацию). Как раз долгий подбор, так как другие ядра очень долго считаются и зачастую на данной задачи дают плохой результат

Удачи в следующих проектах!!!
</div>